In [1]:
import json
import os
import re
import pandas as pd
from collections import Counter
import numpy.random as rand
rand.seed(12345)
from scipy.stats import bernoulli
import numpy as np

In [2]:
length_thr= 510 # as mentioned in DNABert paper

In [3]:
def save_gene_data_json(data):
    with open('all_chrm_genes.json', 'w') as f:
        json.dump(data, f)
        
def load_json_data(file):
    with open(file, 'r') as f:
        data = json.loads(f.read())
    return data

def load_large_json_data(file, num_lines):
    with open(file, 'r') as f:
        data = json.loads(f.read())
    return data

In [4]:
data_base_dir = 'data/'
folder = 'GCF_000001405.39' # this file is obtained from the NCBI website https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.39/ and it contains the Human Reference Genome.

files = [f"chr{id}.fna" for id in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 'MT', 'X', 'Y']]

os.makedirs(os.path.join(data_base_dir, 'cleaned'), exist_ok=True)

In [5]:
print(files)

['chr1.fna', 'chr2.fna', 'chr3.fna', 'chr4.fna', 'chr5.fna', 'chr6.fna', 'chr7.fna', 'chr8.fna', 'chr9.fna', 'chr10.fna', 'chr11.fna', 'chr12.fna', 'chr13.fna', 'chr14.fna', 'chr15.fna', 'chr16.fna', 'chr17.fna', 'chr18.fna', 'chr19.fna', 'chr20.fna', 'chr21.fna', 'chr22.fna', 'chrMT.fna', 'chrX.fna', 'chrY.fna']


In [6]:
choose= bernoulli.rvs(p=0.5)
print(choose)

1


In [7]:
gene_seq_less_thr, chrom_id_list= list(), list()
total_genes_cnt= 0

for chrom_idx, chrom_id in enumerate(files):
    
    # extracting the complete sequence length 
    print('Gene extraction from chromosome file in progress ...')
    print('File to be mapped: ', files[chrom_idx], ' chromosome to be mapped: ', chrom_id)
    
    choose= bernoulli.rvs(p=0.5)
    greater_count= 0
    
    with open(os.path.join(data_base_dir, folder,files[chrom_idx]), "r") as f:
        lines = f.readlines()
        lines = lines[1:]
        all_lines= ''.join(lines)
        all_lines= re.sub('\n', '', all_lines)
        all_lines=re.sub('N', '', all_lines)
        
    start_idx= rand.randint(0, min(1000, len(all_lines)))
    starting= start_idx
    gene_end_idx= len(all_lines) - 1
    
    while starting < gene_end_idx:        
        if bernoulli.rvs(p=0.5) == 0:
            gene_len_chosen= length_thr
        else:
            gene_len_chosen= rand.randint(5,length_thr)

        final_seq= all_lines[starting: min((starting + gene_len_chosen+1), gene_end_idx)]
        gene_seq_less_thr.append(final_seq)
        chrom_id_list.append(chrom_id)

        starting= starting + gene_len_chosen + 1  

Gene extraction from chromosome file in progress ...
File to be mapped:  chr1.fna  chromosome to be mapped:  chr1.fna
Gene extraction from chromosome file in progress ...
File to be mapped:  chr2.fna  chromosome to be mapped:  chr2.fna
Gene extraction from chromosome file in progress ...
File to be mapped:  chr3.fna  chromosome to be mapped:  chr3.fna
Gene extraction from chromosome file in progress ...
File to be mapped:  chr4.fna  chromosome to be mapped:  chr4.fna
Gene extraction from chromosome file in progress ...
File to be mapped:  chr5.fna  chromosome to be mapped:  chr5.fna
Gene extraction from chromosome file in progress ...
File to be mapped:  chr6.fna  chromosome to be mapped:  chr6.fna
Gene extraction from chromosome file in progress ...
File to be mapped:  chr7.fna  chromosome to be mapped:  chr7.fna
Gene extraction from chromosome file in progress ...
File to be mapped:  chr8.fna  chromosome to be mapped:  chr8.fna
Gene extraction from chromosome file in progress ...
Fil

In [8]:
# Only considering genes with length less than 510, results in 14277 genes
print(len(gene_seq_less_thr))

# with the inclusion of 'N', # of gene sequences is 5.2 million (5201879) only 49 sequences had 'N' in it

7641040


In [9]:
print(Counter(chrom_id_list))

Counter({'chr2.fna': 625922, 'chr1.fna': 599559, 'chr3.fna': 515048, 'chr4.fna': 493437, 'chr5.fna': 471516, 'chr6.fna': 442262, 'chr7.fna': 413565, 'chrX.fna': 402908, 'chr8.fna': 377264, 'chr11.fna': 349876, 'chr10.fna': 346282, 'chr12.fna': 346083, 'chr9.fna': 316913, 'chr13.fna': 254954, 'chr14.fna': 235561, 'chr15.fna': 220047, 'chr17.fna': 215798, 'chr16.fna': 212684, 'chr18.fna': 208094, 'chr20.fna': 166363, 'chr19.fna': 152041, 'chr21.fna': 104177, 'chr22.fna': 101872, 'chrY.fna': 68772, 'chrMT.fna': 42})


## Converting gene sequence to k-mer form

In [10]:
# Taken from DNAbert's codebase https://github.com/jerryji1993/DNABERT - motif/motif_utils.py
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space

    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [11]:
# remove duplicates from a list
gene_seq_less_thr_set = list(set(gene_seq_less_thr))
print(len(gene_seq_less_thr)- len(gene_seq_less_thr_set))

9377


In [12]:
# pretrain_seq
print(len(gene_seq_less_thr_set))

# divide the data into 90% training, 10% validation data
choose_seq= bernoulli.rvs(size= len(gene_seq_less_thr_set), p=0.9)

7631663


In [13]:
choose_seq[:100]

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
train_pre_data= open('dnabert_original_train_pretraining_data.txt', 'w')
valid_pre_data= open('dnabert_original_valid_pretraining_data.txt', 'w')

In [15]:
for gene_seq_id, gene_seq in enumerate(gene_seq_less_thr_set):
    kmer_form= seq2kmer(gene_seq, 6)
    
    if choose_seq[gene_seq_id] == 1: # use it for training
        train_pre_data.write(kmer_form + '\n')
    else: # use it for evaluation
        valid_pre_data.write(kmer_form + '\n')   


In [16]:
train_pre_data.close()
valid_pre_data.close()